In [1]:
import sys
sys.path.append('..')

from demo.driver import drive
from demo.shared_params import defaults as params
from demo.runenv import Simulation

In [2]:
params.steps = 'all'
params.one_time_pad_length = 64
params.num_users = 50
params.num_user_log_per_hour = 1
params.num_patient_zeros = 1
params.basedir='../demo'
params.num_days=5
params.debug=False

In [3]:
log = drive(params)

INFO:root:Removing: ../demo/configs
INFO:root:Removing: ../demo/dbs
INFO:root:Removing: ../demo/keys
INFO:root:running simulation


In [4]:
log[0]

{'day': 0,
 'xpos': array([[ 17,   4,  31, ..., 184, 119, 160],
        [443, 451, 518, ..., 902, 882, 760],
        [484, 610, 525, ..., 922, 906, 899],
        ...,
        [861, 884, 806, ..., 932, 925, 980],
        [327, 325, 429, ..., 633, 579, 453],
        [219, 117, 220, ..., 172, 114, 137]]),
 'ypos': array([[732, 741, 764, ..., 901, 977, 913],
        [864, 883, 834, ..., 547, 492, 469],
        [418, 512, 557, ..., 585, 544, 506],
        ...,
        [754, 790, 715, ..., 968, 892, 793],
        [239, 354, 351, ..., 962, 952, 977],
        [677, 853, 865, ..., 560, 426, 365]]),
 'new_diagnosed_patients': [0],
 'users_notified': [0],
 'new_contacts': defaultdict(list, {}),
 'num_results_responsible': 25,
 'num_publishes_for_other_location_auths': 0,
 'num_notifications': 25}

In [5]:
from IPython.display import HTML
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import matplotlib.colors as mcolors
tcolors = list(mcolors.TABLEAU_COLORS)
ncol = len(tcolors)
def get_color(i):
    j = i % ncol
    yield tcolors[j]

def get_colors(n):
    return [next(get_color(i)) for i in range(n)]

In [8]:
import matplotlib.animation

num_entries_per_user_per_day = params.num_user_log_per_hour * 24 + 1
num_entries_per_day = num_entries_per_user_per_day * params.num_users 
asize = num_entries_per_user_per_day * num_entries_per_day
fig, ax = plt.subplots()

x, y = np.zeros(num_entries_per_day*params.num_days), np.zeros(num_entries_per_day*params.num_days)
colors = get_colors(params.num_users) * num_entries_per_user_per_day*params.num_days

sc = ax.scatter(x, y, c=colors)

plt.xlim(params.xmin, params.xmax)
plt.ylim(params.xmin, params.xmax)

def animate(i):
    a, b = [], []
    day = int(i / num_entries_per_user_per_day)
    index = i % num_entries_per_user_per_day
    for j in range(params.num_users):
        a += [log[day]['xpos'][j][index]]
        b += [log[day]['ypos'][j][index]]
    sc.set_offsets(np.c_[a, b])
    
ani = matplotlib.animation.FuncAnimation(
    fig, animate, interval=100, repeat=False,
    frames=params.num_days*num_entries_per_user_per_day
)

plt.close()
HTML(ani.to_html5_video())


INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
INFO:matplotlib.animation:MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 432x288 -pix_fmt rgba -r 10.0 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y /var/folders/g6/86lb24kx1879zx8dzdzbn1pm0000gn/T/tmpk_2o_icl/temp.m4v
